# Redux Dog v Cat

In [1]:
%matplotlib inline
path = "data/redux/"
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
batch_size = 64

TODO:
1. Create validation set and sample
2. Move dog and cat pictures to separate dirs for each newly created directory
3. Finetune and Train
4. Submit to Kaggle

## Create validation set and sample

In [3]:
%cd data/redux/

/home/ubuntu/courses/deeplearning1/nbs/data/redux


In [4]:
%cd train

/home/ubuntu/courses/deeplearning1/nbs/data/redux/train


In [5]:
%mkdir ../valid
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

mkdir: cannot create directory ‘../valid’: File exists
mkdir: cannot create directory ‘../sample’: File exists
mkdir: cannot create directory ‘../sample/train’: File exists
mkdir: cannot create directory ‘../sample/valid’: File exists


In [6]:
from shutil import copyfile

In [7]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
# Move 2000 data to validation set
# Copy 50 to create a sample validation set
for i in range(50): 
    copyfile(shuf[i], '../sample/valid/' + shuf[i])
    os.rename(shuf[i], '../valid/' + shuf[i])

for i in range(50, 2000):
    os.rename(shuf[i], '../valid/' + shuf[i])

# Copy 200 data disjoint from the sample validation set
# to create a sample training set
for i in range(2000, 2200):
    copyfile(shuf[i], '../sample/train/' + shuf[i])

In [12]:
%cd ..

/home/ubuntu/courses/deeplearning1/nbs/data/redux


In [14]:
# Check that the newly created direcotires have the desired nubmer of data
%ls -U sample/train/ | wc -l
%ls -U sample/valid/ | wc -l
%ls -U valid/ | wc -l

200
50
2000


## Move dog and cat pictures to separate dirs for each newly created directory

In [4]:
%cd /home/ubuntu/courses/deeplearning1/nbs/data/redux/train

/home/ubuntu/courses/deeplearning1/nbs/data/redux/train


In [20]:
%mkdir cats dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [35]:
%cd ../sample/train/

/home/ubuntu/courses/deeplearning1/nbs/data/redux/sample/train


In [36]:
%mkdir cats dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [38]:
cd ../valid/

/home/ubuntu/courses/deeplearning1/nbs/data/redux/sample/valid


In [40]:
%mkdir cats dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [41]:
cd ../

/home/ubuntu/courses/deeplearning1/nbs/data/redux/sample


In [42]:
cd ../valid/

/home/ubuntu/courses/deeplearning1/nbs/data/redux/valid


In [43]:
%mkdir cats dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [20]:
cd ../test/

/home/ubuntu/courses/deeplearning1/nbs/data/redux/test


In [21]:
%mkdir unknown
%mv *.jpg unknown/

In [53]:
cd ..

/home/ubuntu/courses/deeplearning1/nbs/data/redux


In [55]:
cd ..

/home/ubuntu/courses/deeplearning1/nbs/data


In [56]:
cd ..

/home/ubuntu/courses/deeplearning1/nbs


## Finetune and train

In [5]:
cd ~/courses/deeplearning1/nbs

/home/ubuntu/courses/deeplearning1/nbs


In [6]:
from vgg16 import Vgg16

In [7]:
vgg = Vgg16()

In [8]:
batches = vgg.get_batches(path + 'train', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'valid', batch_size = batch_size * 2)
vgg.finetune(batches)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [58]:
vgg.fit(batches, val_batches, nb_epoch = 1)

Epoch 1/1
23000/23000 [==============================] - 605s - loss: 0.1237 - acc: 0.9674 - val_loss: 0.0537 - val_acc: 0.9860


In [66]:
%mkdir 'data/redux/results'

mkdir: cannot create directory ‘data/redux/results’: File exists


In [67]:
vgg.model.save_weights(path + 'results/ft1.h5')

## Run a few more epoches

In [9]:
cd ~/courses/deeplearning1/nbs

/home/ubuntu/courses/deeplearning1/nbs


In [ ]:
vgg.fit(batches, val_batches, nb_epoch = 1)

Epoch 1/1
  896/23000 [>.............................] - ETA: 547s - loss: 0.4602 - acc: 0.8661

## Submit

In [25]:
batches, preds = vgg.test(path + 'test', batch_size = batch_size * 2)

Found 12500 images belonging to 1 classes.


In [26]:
filenames = batches.filenames

In [28]:
preds[:5]

array([[  1.0000e+00,   5.1835e-10],
       [  9.4557e-01,   5.4426e-02],
       [  1.0000e+00,   4.0396e-06],
       [  1.0000e+00,   2.8716e-13],
       [  2.8611e-10,   1.0000e+00]], dtype=float32)

In [34]:
filenames[:5]

['unknown/9292.jpg',
 'unknown/12026.jpg',
 'unknown/9688.jpg',
 'unknown/4392.jpg',
 'unknown/779.jpg']

In [ ]:
save_array(path + 'results/test_pred.dat', preds)
save_array(path + 'results/filenames.dat', filenames)

In [ ]:
preds = load_array(path + 'results/test_pred.dat')
